# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "demo_human_or_worm"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

demo_human_or_worm 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

2021-11-23 01:00:57.642175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-23 01:00:57.642195: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2021-11-23 01:01:01.271868: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-23 01:01:01.271905: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michle): /proc/driver/nvidia/version does not exist
2021-11-23 01:01:01.272552: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Done.
Unzipping...

Done.


In [5]:
info(DATASET)

Dataset `demo_human_or_worm` has 2 classes: human, worm.

All lenghts of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.


,train,test
human,37500,12500
worm,37500,12500


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 75000 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
# vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-11-23 01:01:20.892629: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 'a', 't', 'c', 'g', 'n']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


   1/1172 [..............................] - ETA: 23:11 - loss: 0.7378 - binary_accuracy: 0.4688 - f1_score: 0.3111

   5/1172 [..............................] - ETA: 15s - loss: 0.7177 - binary_accuracy: 0.4750 - f1_score: 0.1692  

   9/1172 [..............................] - ETA: 17s - loss: 0.6980 - binary_accuracy: 0.5295 - f1_score: 0.1507

  13/1172 [..............................] - ETA: 17s - loss: 0.6823 - binary_accuracy: 0.5577 - f1_score: 0.2196

...

1163/1172 [============================>.] - ETA: 0s - loss: 0.1343 - binary_accuracy: 0.9473 - f1_score: 0.9437

1166/1172 [============================>.] - ETA: 0s - loss: 0.1342 - binary_accuracy: 0.9473 - f1_score: 0.9438

1169/1172 [============================>.] - ETA: 0s - loss: 0.1342 - binary_accuracy: 0.9473 - f1_score: 0.9438

1172/1172 [==============================] - ETA: 0s - loss: 0.1341 - binary_accuracy: 0.9473 - f1_score: 0.9438

1172/1172 [==============================] - 29s 24ms/step - loss: 0.1341 - binary_accuracy: 0.9473 - f1_score: 0.9438


## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 25000 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

  1/391 [..............................] - ETA: 2:01 - loss: 0.2657 - binary_accuracy: 0.9062 - f1_score: 0.8788

  9/391 [..............................] - ETA: 2s - loss: 0.1904 - binary_accuracy: 0.9167 - f1_score: 0.9014  

 17/391 [>.............................] - ETA: 2s - loss: 0.1829 - binary_accuracy: 0.9265 - f1_score: 0.9063

 28/391 [=>............................] - ETA: 2s - loss: 0.1832 - binary_accuracy: 0.9308 - f1_score: 0.9110

 40/391 [==>...........................] - ETA: 1s - loss: 0.1687 - binary_accuracy: 0.9336 - f1_score: 0.9140

...

368/391 [===========================>..] - ETA: 0s - loss: 0.1674 - binary_accuracy: 0.9356 - f1_score: 0.9173

376/391 [===========================>..] - ETA: 0s - loss: 0.1673 - binary_accuracy: 0.9357 - f1_score: 0.9173

384/391 [============================>.] - ETA: 0s - loss: 0.1683 - binary_accuracy: 0.9353 - f1_score: 0.9172

391/391 [==============================] - 2s 5ms/step - loss: 0.1683 - binary_accuracy: 0.9353 - f1_score: 0.9170


[0.16827392578125, 0.9353200197219849, 0.916961669921875]